In [38]:
import os, sys, csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import nibabel as nb
from scipy.stats import pearsonr, spearmanr
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


# get scan params

In [2]:
filename  = '/data/pt_mar006/subjects/sd02_d01/nifti/mprage/T1.nii.gz'
T1_image = nb.load(filename)
print T1_image.header

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : 
db_name         : 
extents         : 0
session_error   : 0
regular         : 
dim_info        : 27
dim             : [  3 192 256 256   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [-1.0e+00  1.0e+00  1.0e+00  1.0e+00  1.9e+03  1.0e+00  1.0e+00  1.0e+00]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 18
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : 
aux_file        : 
qform_code      : unknown
sform_code      : aligned
quatern_b       : -0.00509426
quatern_c       : -0.99997383
quatern_d       : -2.6166439e-05
qoffset_x       : 94.50895
qoffset_y       : -98.21346
qoffse

# healthy controls 

In [4]:
# get sample correlation coefficient 

hc_filename = '/data/pt_mar006/documents/all_hc_datasummary.xlsx'

dH = pd.read_excel(hc_filename,
                   index_col = 'study_id')

mean_hc = dH.age.mask(dH.inclusion != 'yes').mean()

std_hc  = dH.age.mask(dH.inclusion !='yes').std()

print "number of healthy subjects included: ", len(dH[dH.inclusion == 'yes']['age'])

print "AGE mean, std: ", mean_hc, std_hc

number of healthy subjects included:  28
AGE mean, std:  65.21428571428571 8.837450373299365


In [6]:
list_name    = '/data/pt_mar006/documents/all_hc_after_qc.txt'
with open(list_name) as f:
    subject_healthy = f.read().splitlines()
len(subject_healthy)

28

In [10]:
dH.ix[subject_healthy].sex

study_id
hc01_d00    m
hc02_d00    m
hc03_d00    m
hc04_d00    m
hc05_d00    m
hc06_d00    f
hc07_d00    f
hc08_d00    f
hc10_d00    f
hc11_d00    m
hc12_d00    f
hc13_d00    m
hc14_d00    m
hc15_d00    f
hc17_d00    f
hc19_d00    m
hc20_d00    f
hc21_d00    f
hc22_d00    m
hc23_d00    f
hc24_d00    m
hc25_d00    m
hc27_d00    f
hc28_d00    m
hc30_d00    f
hc31_d00    m
hc32_d00    m
hc33_d00    f
Name: sex, dtype: object

# stroke patients 

In [19]:
file_beh  = '/data/pt_mar006/documents/all_sd_datasummary_STEIGLITZ.xlsx'

db = pd.read_excel(file_beh, 
                   index_col='study_id') 

len(db)


54

In [21]:
db.age.mean(), db.age.std()

(63.77777777777778, 12.034018865819228)

# stroke patients with three consequtive resting-state and no history

In [16]:
list_name    = '/data/pt_mar006/documents/all_sd_after_qc_nohistory.txt'
with open(list_name) as f:
    subject_list_3rs = f.read().splitlines()
len(subject_list_3rs)

28

In [17]:
print "number of stroke patients included: ", len(db[db.inclusion == 'yes']['age'])
print "AGE mean, std: ", db.ix[subject_list_3rs].age.mean(),  db.ix[subject_list_3rs].age.std()

number of stroke patients included:  28
AGE mean, std:  65.03571428571429 13.270636320832109


In [22]:
db.ix[subject_list_3rs].sex

study_id
sd02    m
sd05    m
sd08    m
sd10    f
sd13    m
sd14    f
sd16    m
sd17    m
sd21    f
sd25    f
sd26    m
sd28    f
sd31    f
sd32    m
sd33    m
sd35    f
sd36    m
sd38    m
sd39    m
sd40    f
sd42    m
sd43    f
sd44    m
sd45    f
sd46    f
sd48    m
sd49    m
sd52    m
Name: sex, dtype: object

# scan day 1

In [23]:
print "first scanning day mean and std: ",
db[db.inclusion == 'yes'].scan_two.mean(), db[db.inclusion == 'yes'].scan_two.std()

first scanning day mean and std: 

(1.0, 0.0)

# scan day 5

In [25]:
print "last scanning day mean and std: ", 
db[db.inclusion == 'yes'].scan_last.mean(), db[db.inclusion == 'yes'].scan_last.std(), 

last scanning day mean and std: 

(4.928571428571429, 0.3779644730092272)

# nihss scores

### nihss admission

In [29]:
print db.ix[subject_list_3rs].NIHSS_admission.mean()
print db.ix[subject_list_3rs].NIHSS_admission.std() 

2.5714285714285716
2.700186158583384


### nihss discharge

In [30]:
print db.ix[subject_list_3rs].NIHSS_discharge.mask(db.NIHSS_discharge=='no').mean()
print db.ix[subject_list_3rs].NIHSS_discharge.mask(db.NIHSS_discharge=='no').std()

1.0
1.1094003924504583


# mrs scores

### mrs admission

In [44]:
print db.ix[subject_list_3rs].mRS_admission.mask(db.mRS_admission=='no').mean()
print db.ix[subject_list_3rs].mRS_admission.mask(db.mRS_admission=='no').std()

1.4615384615384615
1.448606757702565


### mrs discharge

In [45]:
print db.ix[subject_list_3rs].mRS_discharge.mask(db.mRS_discharge=='no').mean()
print db.ix[subject_list_3rs].mRS_discharge.mask(db.mRS_discharge=='no').std()

0.7692307692307693
0.9511127086814605


# test demographics between healthy and stroke

### gender diff

In [33]:
df_demog = pd.DataFrame({'study': ['healthy', 'stroke'],
                         'women': [13, 11],
                         'man':[15, 17]})

#df_demog.set_index('study')
df_demog = df_demog.set_index("study")

df_demog

,man,women
study,,
healthy,15,13
stroke,17,11


In [34]:
from scipy import stats

# write 1's for males and 0's for females in both groups

x = [ 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
      0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
     
y  = [ 1.,  1.,  1.,  0,  1.,  0.,  1.,  1.,  0.,  0.,  1., 0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  
      0.,  1.,  0.,  1.,  0., 0.,  1.,  1.,  1. ]

print len(x), len(y)
# H0 : median of all of the groups are equal.
# p is not significant --> don't reject null hypothesis


stats.kruskal(x, y)

28 28


KruskalResult(statistic=0.2864583333333028, pvalue=0.5924991041730836)

### age diff 

In [35]:
print "number of healthy subjects included: ", len(dH[dH.inclusion == 'yes']['age'])
dH[dH.inclusion == 'yes']['age'].values


number of healthy subjects included:  28


array([57, 72, 65, 69, 72, 70, 67, 64, 68, 68, 72, 61, 65, 67, 65, 76, 69,
       64, 65, 69, 66, 63, 64, 71, 66, 69, 56, 26])

In [36]:
print "number of stroke patients included: ", len(db[db.inclusion == 'yes'])

db[db.inclusion == 'yes'].age.values

number of stroke patients included:  28


array([47, 51, 64, 67, 76, 72, 54, 43, 72, 71, 79, 73, 81, 47, 68, 75, 50,
       53, 59, 90, 72, 75, 78, 37, 63, 60, 63, 81])

In [39]:
stats.ttest_ind(dH[dH.inclusion == 'yes']['age'], 
                      db[db.inclusion == 'yes']['age'], 
                      axis=0, equal_var=False, 
                      )


Ttest_indResult(statistic=0.059264469630969964, pvalue=0.9529926703510472)

# stroke patients lesion volumes

In [52]:
# get lesion volumes of patients included
data_dir = '/data/pt_mar006/subjects_masks/'
les_nam  = 'lesion_mask_mni_dilated.nii.gz'
gm_nam   = 'gm_mask_no_lesion.nii.gz'
rLesion = pd.DataFrame(index=subject_list, 
                       columns=['les_count',
                                'les_vol'])
for subject_id in subject_list:
    
    les_file = os.path.join(data_dir, subject_id, les_nam)
    les_mask = nb.load(les_file).get_data()
    les_cnt = len(np.where(les_mask==1)[0])
    
    rLesion.ix[subject_id]['les_count'] = les_cnt
    rLesion.ix[subject_id]['les_vol'] = les_cnt * 3*3*3 * 0.001 # (cm^3)

In [78]:
print "LESION VOLUME MEAN AND STD:"
print rLesion.les_vol.mean(), rLesion.les_vol.std()

LESION VOLUME MEAN AND STD:
4.112678571428572 2.8012423838106444


In [62]:
rLesion

,les_count,les_vol
sd02,192,5.184
sd05,277,7.479
sd08,151,4.077
sd10,135,3.645
sd13,69,1.863
sd14,45,1.215
sd16,75,2.025
sd17,78,2.106
sd21,522,14.094
sd25,252,6.804


# gray matter template used to get gradients from healthy controls

In [40]:
image_mask = os.path.join('/data/pt_mar006/subjects_group',
                          'mni3_rest_gm_mask.nii.gz')

A = nb.load(image_mask).get_data()

In [41]:
print "number of voxels used to get gradients: ",  np.where(A == 1)[0].shape

number of voxels used to get gradients:  (33327,)


# gray matter templates used for individual patients

In [42]:
data_dir         = '/data/pt_mar006/subjects_masks/'
gmask_file       = 'gm_mask_no_lesion.nii.gz'
concordance_file = 'conc_ccc.nii.gz'

list_name    = '/data/pt_mar006/documents/all_sd_after_qc_nohistory.txt'
with open(list_name) as f:
    subject_list = f.read().splitlines()

DF  = pd.DataFrame()
RES = {}

B = []
for subject_id in subject_list:
    
    gm_file     = os.path.join(data_dir, subject_id, 
                               gmask_file)

    tmp = nb.load(gm_file).get_data()
    length = len(np.where(tmp==1)[0])
    print length
    B.append(length)

33134
33086
33177
32888
33179
32935
33185
32730
33110
32760
33015
33070
32925
32659
33114
33040
32768
33116
33085
32710
33119
33212
33005
33155
33146
33201
33122
33162


In [43]:
B = np.array(B)
print "patients with min and max GM voxels: ", B.min(), B.max()

patients with min and max GM voxels:  32659 33212


# vizualize table

In [79]:
table = pd.concat([db.ix[subject_list].age,
                   db.ix[subject_list].sex,
                   rLesion.ix[subject_list].les_vol,
                   db.ix[subject_list].NIHSS_admission,
                   db.ix[subject_list].NIHSS_discharge,
                   db.ix[subject_list].mRS_admission,
                   db.ix[subject_list].mRS_discharge,
                   ],
                   axis =1)

In [80]:
table

,age,sex,les_vol,NIHSS_admission,NIHSS_discharge,mRS_admission,mRS_discharge
study_id,,,,,,,
sd02,47,m,5.184,6,1,2,1
sd05,51,m,7.479,5,3,2,2
sd08,64,m,4.077,1,0,1,0
sd10,67,f,3.645,0,0,0,0
sd13,76,m,1.863,7,0,4,0
sd14,72,f,1.215,0,0,0,0
sd16,54,m,2.025,1,0,0,0
sd17,43,m,2.106,3,2,2,2
sd21,72,f,14.094,1,0,3,2


In [45]:
## Create a Pandas Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter('/data/pt_mar006/documents/table_tmp.xlsx')
#table.ix[subject_list].to_excel(writer, sheet_name='Sheet1')
#writer.close()

# # get number of voxels for each patient used to compute concordance

In [16]:
mask_dir = '/data/pt_mar006/subjects_masks/'
df       = pd.DataFrame(index=subject_list, 
                        columns=['voxnum'])

for sbj in subject_list:
    sbj_mask = os.path.join(mask_dir, sbj, 'gm_mask.nii.gz')
    mask_arr = nb.load(sbj_mask).get_data()
    voxnum   = np.where(mask_arr == 1)[0].shape
    df.ix[sbj]['voxnum'] = voxnum

In [17]:
print df.voxnum.min(), df.voxnum.max()

(32745,) (33271,)


In [18]:
print "variation in voxel numbers: ",  df.voxnum.max()[0] - df.voxnum.min()[0]

variation in voxel numbers:  526
